In [2]:
!pip install pandas
!pip install requests
!pip install lxml

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.4/11.5 MB 12.9 MB/s eta 0:00:01
   --- ------------------------------------ 0.9/11.5 MB 11.9 MB/s eta 0:00:01
   ----- ---------------------------------- 1.5/11.5 MB 12.0 MB/s eta 0:00:01
   ------- -------------------------------- 2.0/11.5 MB 11.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.5/11.5 MB 11.2 MB/s eta 0:00:01
   ---------- ----------------------------- 3.1/11.5 MB 11.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.5 MB 11.5 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.5 MB 11.5 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.5 MB 11.1 MB/s eta 0:00

In [18]:
import pandas as pd
import requests
from lxml import html as lxml_html
import re
import csv

# requests.get 날릴 base_rul
# base_url = f"http://www.law.go.kr/DRF/lawSearch.do?target=prec&OC=hsk9174&type=HTML&display=100&page={}&org=400201&JO=형법"

# 판례 ID받아올 리스트
ID_list = []

# ID가 담긴 하이퍼링크 받아오는 코드 (총 75페이지)

for page in range(1,75):
    url = f"http://www.law.go.kr/DRF/lawSearch.do?target=prec&OC=hsk9174&type=HTML&display=100&page={page}&org=400201&JO=형법"
    response = requests.get(url)

    # html로 받았으므로, html로 파싱
    tree = lxml_html.fromstring(response.content)

    # XPath 주소로 하이퍼링크 주소에 접근하기
    addresses = tree.xpath("/html/body/form/div/table/tbody/tr/td/a/@href")

    # 해당 주소가 ID_number라는 리스트로 담김. 
    # 해당 리스트 요소에서 정규표현식을 통해 숫자 제외 전부 제외
    for address in addresses:
        match = re.search(r'ID=(\d+)', address)
        ID = match.group(1)
        ID_list.append(ID)

# 생성된 ID_list에 관하여 CSV파일로 바로 저장
with open('ID_list.csv', 'w', newline ='') as file:
    writer = csv.writer(file)
    writer.writerow(ID_list)

In [19]:
import csv

# 리스트를 초기화합니다
data = []

# CSV 파일을 엽니다
with open('/home/kannakawai9/NLP_single_project/law_data_dl/data_crawling/ID_list.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.extend(row)

print(data)

['239459', '239467', '238463', '239243', '239199', '239227', '239229', '239231', '239237', '239247', '238861', '238867', '239263', '238907', '238931', '238933', '238025', '238027', '238021', '237915', '237941', '237943', '237865', '237547', '237581', '237597', '237217', '238949', '238945', '238643', '238637', '238961', '238979', '236821', '238991', '236931', '238997', '239003', '235561', '235559', '235557', '235555', '238999', '235553', '239005', '235565', '239009', '235563', '235107', '239011', '239013', '238405', '234985', '238411', '238409', '238413', '234935', '234803', '234713', '234701', '234715', '234709', '234707', '234711', '238421', '234599', '234453', '234443', '238221', '234401', '234321', '238227', '234335', '233983', '233981', '238231', '238235', '233887', '233889', '238233', '233885', '238239', '233581', '238241', '233587', '238253', '238249', '238247', '233583', '233585', '238251', '234057', '238263', '233091', '233095', '233045', '233093', '233053', '233049', '232763',

In [20]:
len(data)

7346

In [ ]:
base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

for ID in data[:2]:
    
    url = base_url + str(ID)
    
    response = requests.get(url)

    tree = lxml_html.fromstring(response.content)

    # 판례 ID:
    case_ids = ID

    # 판시사항: 
    case_details = tree.xpath('//*[@id="conScroll"]/p[1]')

    # 판결요지: 
    judgment_summaries = tree.xpath('//*[@id="conScroll"]/p[2]')

    # 참조조문: 
    referenced_statutes = tree.xpath('//*[@id="conScroll"]/p[3]/a')

    # 참조판례: 
    referenced_precedents = tree.xpath('//*[@id="conScroll"]/p[4]')

    # 변호인: 
    attorneys = tree.xpath('//*[@id="conScroll"]/p[7]')

    # 주문: 
    orders = tree.xpath('//*[@id="conScroll"]/p[9]')

    # 이유: 
    reasons = tree.xpath('//*[@id="conScroll"]/p[10]')
    
    # 대법관: 
    judges = tree.xpath('//*[@id="conScroll"]/div')

    
# Create a DataFrame from the lists
df = pd.DataFrame({
    'Case ID': [case_ids],
    'Case Details': [case_details],
    'Judgment Summary': [judgment_summaries],
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})
df

In [56]:
case_details

[' [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단하는 기준 / 외부인의 출입이 엄격히 통제되는 건조물에 거주자나 관리자의 승낙 없이 몰래 들어간 경우 또는 출입 당시 거주자나 관리자가 출입의 금지나 제한을 하였음에도 이를 무시하고 출입한 경우, 침입행위에 해당할 수 있는지 여부',
 ' [2] 피고인이 ‘甲에게 100m 이내로 접근하지 말 것’ 등을 명하는 법원의 접근금지가처분 결정이 있는 등 피고인이 甲을 방문하는 것을 甲이 싫어하는 것을 알고 있음에도 임의로 甲이 근무하는 사무실 안으로 들어감으로써 건조물에 침입하였다는 공소사실로 기소된 사안에서, 피고인이 위 결정에 반하여 甲이 근무하는 사무실에 출입한 것은 甲의 명시적인 의사에 반하는 행위일 뿐만 아니라, 출입의 금지나 제한을 무시하고 출입한 경우로서 출입 당시 객관적·외형적으로 드러난 행위태양을 기준으로 보더라도 사실상 평온상태가 침해된 것으로 볼 수 있으므로 건조물침입죄가 성립한다고 한 사례']

In [ ]:
df = pd.DataFrame({
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})
df

In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import requests
from lxml import html as lxml_html
from tqdm import tqdm

base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

# Initialize empty lists for each column
case_ids = []
case_details = []
judgment_summaries = []
referenced_statutes = []
referenced_precedents = []
attorneys = []
orders = []
reasons = []
judges = []

for ID in tqdm(data):
    url = base_url + str(ID)
    response = requests.get(url)
    tree = lxml_html.fromstring(response.content)

    # 요소 추가
    case_ids.append(ID)
    case_details.append(tree.xpath('//*[@id="conScroll"]/p[1]/text()'))
    judgment_summaries.append(tree.xpath('//*[@id="conScroll"]/p[2]/text()'))
    referenced_statutes.append(tree.xpath('//*[@id="conScroll"]/p[3]/a/text()'))
    referenced_precedents.append(tree.xpath('//*[@id="conScroll"]/p[4]/text()'))
    
    orders.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-1]/text()'))
    reasons.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()]/text()'))
    judges.append(tree.xpath('//*[@id="conScroll"]/div/text()'))
    if '변호사' in tree.xpath('//*[@id="conScroll"]/p[7]/text()')[0]:
        attorneys.append(tree.xpath('//*[@id="conScroll"]/p[7]/text()'))
    else:
        attorneys.append([])

# 딕셔너리로 리스트를 요소로 넣어 데이터프레임 만들기
df = pd.DataFrame({
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})

df


In [ ]:
df.to_csv('case.csv')

In [ ]:
!pip install aiohttp

In [ ]:
!pip install numpy

In [ ]:
!pip install tqdm

In [21]:
import pandas as pd
import asyncio
import aiohttp
from lxml import html as lxml_html
from tqdm import tqdm
import nest_asyncio
import numpy as np

# Apply nest_asyncio to enable nested running of asyncio
nest_asyncio.apply()

base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

# Initialize empty lists for each column
case_info = []
case_ids = []
case_details = []
judgment_summaries = []
referenced_statutes = []
referenced_precedents = []
attorneys = []
orders = []
reasons = []
judges = []

async def fetch(ID, session):
    url = base_url + str(ID)
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        for i in tqdm(range(0, len(data), 40)):
            tasks = []
            for ID in data[i:i+40]:
                tasks.append(fetch(ID, session))
            htmls = await asyncio.gather(*tasks)
            for ID, html in zip(data[i:i+40], htmls):
                tree = lxml_html.fromstring(html)
                # Append the extracted data to the respective lists
                case_info.append(tree.xpath('//*[@id="contentBody"]/div[1]/text()'))
                case_ids.append(ID)

                # 판시사항
                case_details.append(tree.xpath('//*[@id="conScroll"]/p[1]/text()'))

                # 판결요지 있다면
                if tree.xpath('//*[@id="yo"]/text()'):
                     judgment_summaries.append(tree.xpath('//*[@id="conScroll"]/p[2]/text()'))
                     
                     # 참조조문은 p[3]
                     referenced_statutes.append(tree.xpath('//*[@id="conScroll"]/p[3]//text()'))

                     # 참조판례도 있다면
                     if tree.xpath('//*[@id="conPrec"]/text()'):
                         referenced_precedents.append(tree.xpath('//*[@id="conScroll"]/p[4]//text()'))

                     # 참조판례가 없다면
                     else:
                         referenced_precedents.append(np.nan)
                         

                # 만약 판결요지가 없다면
                elif not tree.xpath('//*[@id="yo"]/text()'):
                     judgment_summaries.append(np.nan)
                     # 참조조문은 p[2]
                     referenced_statutes.append(tree.xpath('//*[@id="conScroll"]/p[2]//text()'))

                     # 그런데 참조판례는 있다면
                     if tree.xpath('//*[@id="conPrec"]/text()'):
                         referenced_precedents.append(tree.xpath('//*[@id="conScroll"]/p[3]//text()'))

                     # 참조판례가 없다면
                     else:
                         referenced_precedents.append(np.nan)

                orders.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-1]/text()'))
                reasons.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()]/text()'))
                judges.append(tree.xpath('//*[@id="conScroll"]/div/text()'))
                attorney_text = tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-3]/text()')
                keywords = ['변호사', '법무', '법인', '외']
                if attorney_text and any(keyword in ''.join(attorney_text) for keyword in keywords):
                    attorneys.append(attorney_text)
                else:
                    attorneys.append(np.nan)
            # Wait for 5 seconds
            await asyncio.sleep(5)

# Run the main function
await main()


# Create a DataFrame from the lists
df = pd.DataFrame({
    'Case Info' : case_info,
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})

df


100%|██████████| 184/184 [19:14<00:00,  6.27s/it]


,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge
0,[\n [대법원 2024. 2. 15. 선고 2023도15164 판결]...,239459,"[ [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입죄...",[ [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의 ...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항, ...","[ [1] , 대법원 2009. 9. 10. 선고 2009도4335 판결, (공20...",NaN,"[ 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 피고인은, ① 2...",[\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ]
1,[\n [대법원 2024. 2. 8. 선고 2023도16595 판결]\...,239467,[ [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단...,[ [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성요...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항]","[ [1] , 대법원 2022. 3. 24. 선고 2017도18272 전원합의체 판...",[ 변호사 장원철],"[ 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을 ...","[ 상고이유를 판단한다. , 1. 2021. 9. 7. 건조물침입 부분 공소사실...",[\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...
2,[\n [대법원 2024. 1. 25. 선고 2020도10330 판결]...,238463,[ 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당사...,[ 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 재...,[형법 제347조 제1항],"[대법원 2004. 6. 25. 선고 2003도7124 판결, (공2004하, 12...",[ 법무법인 청률 담당변호사 김태창],"[ 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 공소외인은 피고인 ...",[\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...
3,[\n [대법원 2024. 1. 4. 선고 2022도699 판결]\n ...,239243,"[ [1] , 명예훼손죄의 구성요건 중 비방할 목적이 있는지와 드러낸 사실이 사회...","[ [1] , 은 “사람을 비방할 목적으로 정보통신망을 통하여 공공연하게 사실을 드...","[ [1] , 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항, ,...","[ [1][2] , 대법원 2020. 12. 10. 선고 2020도11471 판결,...",[ 법무법인 별 외 5인],[ 상고를 모두 기각한다.],"[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. ‘Bad Fath...",[\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ]
4,[\n [대법원 2024. 1. 4. 선고 2023도11313 판결]\...,239199,"[ [1] 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 , 의 공기호에 해당하...",[ [1] 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용하...,"[ [1] , 형법 제238조, , [2] , 형법 제238조]","[ [1] , 대법원 1995. 9. 5. 선고 95도1269 판결, (공1995하...",[ 법무법인 더킴로펌 담당변호사 김형석 외 1인],"[ 원심판결을 파기하고, 사건을 창원지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. 피고인은 2020...",[\n 대법관 이동원(재판장) 천대엽 권영준(주심) \n ]
...,...,...,...,...,...,...,...,...,...,...
7341,[\n [대법원 1953. 11. 23. 선고 4286형상162 판결]...,215365,[ 군정법령 제5호의 내용이 신형법과 저촉되는 경우],"[ 폭발물 소지에 관하여는 , 와 군정법령 제5호 제2조와의 내용이 동일하므로 그 ...","[ 군정법령제5호 제2조, , 형법 제121조, , , 국가보안법 제3조]",NaN,NaN,[ 원판결을 파훼하고 본건을 서울고등법원에 환송한다.],"[ 피고인의 변호인 정근영 상고취의는 원심판결은 , 소정의 이유로서 파훼를 불면일...",[\n 대법관 김병로(재판장) 김세완 김갑수 허진 배정현 \n ...
7342,[\n [대법원 1953. 11. 9. 선고 4286형상181 판결]\...,215291,"[가. 간첩예비와 의율, 나. 간첩행위의 해석, 다. 군법회의 확정판결의 해석]","[가. 간첩예비죄에 , 만을 인용하여도 , 는 그 내용이된 것이므로 동 법조를 인용...","[국가보안법 제3조, , , 구 형법 제85조 제1항, , , 제87조, , , 제...",NaN,NaN,[ 본건 상고를 기각한다.],"[피고인 1, 피고인 2, 피고인 3에 대한 검사 김홍수의 상고취의는 원판결은 피고...",[\n 대법관 김병로(재판장) 김세완 김갑수 허진 고재호 \n ...
7343,[\n [대법원 1952. 5. 27. 선고 4284형상76 판결]\n...,188581,[ 행위시와 재판시간에 수차 법령의 개폐에 의한 형의 변경과 법령 적용],[ 범죄행위 당시와 재판시의 중간에 수차 법령의 개폐로 인하여 형의 변경이 있는 때...,"[형법 제6조, , , , , 제10조, , 법령제127호 제2조 처벌, 법률 ...",NaN,NaN,[ 원판결을 파훼하고 본건을 서울고등법원에 차려한다.],[ 검사 박팔천의 상고이유는 원판결은 「범죄사실 급 증거설시는 원판결 적시사실과 동...,[]
7344,[\n [대법원 1951. 5. 1. 선고 4283형상73 판결]\n ...,85840,[ 방화살인과 견련범],[에 소위 범죄의 수단이라 함은 어느 범죄의 성질상 그 수단으로 보통 사용되는 행위...,"[형법 제54조, , , , 제108조, , , , 제198조]",NaN,NaN,[ 원판결은 파훼하고 본건을 서울고등법원에 차려하다.],[ 피고인 1의 변호인 양병오의 상고이유는 원판결은 의율착오의 위법을 면치 못할 것...,[]


In [22]:
df.to_csv('case_data.csv', index=False)

In [23]:
df.head()

,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge
0,[\n [대법원 2024. 2. 15. 선고 2023도15164 판결]...,239459,"[ [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입죄...",[ [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의 ...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항, ...","[ [1] , 대법원 2009. 9. 10. 선고 2009도4335 판결, (공20...",NaN,"[ 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 피고인은, ① 2...",[\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ]
1,[\n [대법원 2024. 2. 8. 선고 2023도16595 판결]\...,239467,[ [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단...,[ [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성요...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항]","[ [1] , 대법원 2022. 3. 24. 선고 2017도18272 전원합의체 판...",[ 변호사 장원철],"[ 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을 ...","[ 상고이유를 판단한다. , 1. 2021. 9. 7. 건조물침입 부분 공소사실...",[\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...
2,[\n [대법원 2024. 1. 25. 선고 2020도10330 판결]...,238463,[ 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당사...,[ 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 재...,[형법 제347조 제1항],"[대법원 2004. 6. 25. 선고 2003도7124 판결, (공2004하, 12...",[ 법무법인 청률 담당변호사 김태창],"[ 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 공소외인은 피고인 ...",[\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...
3,[\n [대법원 2024. 1. 4. 선고 2022도699 판결]\n ...,239243,"[ [1] , 명예훼손죄의 구성요건 중 비방할 목적이 있는지와 드러낸 사실이 사회...","[ [1] , 은 “사람을 비방할 목적으로 정보통신망을 통하여 공공연하게 사실을 드...","[ [1] , 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항, ,...","[ [1][2] , 대법원 2020. 12. 10. 선고 2020도11471 판결,...",[ 법무법인 별 외 5인],[ 상고를 모두 기각한다.],"[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. ‘Bad Fath...",[\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ]
4,[\n [대법원 2024. 1. 4. 선고 2023도11313 판결]\...,239199,"[ [1] 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 , 의 공기호에 해당하...",[ [1] 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용하...,"[ [1] , 형법 제238조, , [2] , 형법 제238조]","[ [1] , 대법원 1995. 9. 5. 선고 95도1269 판결, (공1995하...",[ 법무법인 더킴로펌 담당변호사 김형석 외 1인],"[ 원심판결을 파기하고, 사건을 창원지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. 피고인은 2020...",[\n 대법관 이동원(재판장) 천대엽 권영준(주심) \n ]


In [14]:
print(
    'Case Info', len(case_info),
    'Case ID', len(case_ids),
    'Case Details', len(case_details),
    'Judgment Summary', len(judgment_summaries),
    'Referenced Statutes', len(referenced_statutes),
    'Referenced Precedents', len(referenced_precedents),
    'Attorney', len(attorneys),
    'Order', len(orders),
    'Reason', len(reasons),
    'Judge', len(judges)
)



Case Info 7346 Case ID 7346 Case Details 7346 Judgment Summary 7346 Referenced Statutes 7346 Referenced Precedents 7346 Attorney 7346 Order 7346 Reason 7346 Judge 7346


In [29]:
df.isnull().sum()

Case Info                0
Case ID                  0
Case Details             0
Judgment Summary         7
Referenced Statutes      0
Referenced Precedents    5
Attorney                 4
Order                    0
Reason                   0
Judge                    0
dtype: int64

In [30]:
null_cases = df[(df['Attorney'].isnull())]
case_ids_with_nulls = null_cases['Case ID']
print(case_ids_with_nulls)


0     239459
5     239227
8     239237
18    238021
Name: Case ID, dtype: object


In [ ]:
null_cases = df[(df['Judgment Summary'].isnull()) & (df['Referenced Precedents'].isnull())]
case_ids_with_nulls = null_cases['Case ID']
print(case_ids_with_nulls)


In [35]:
import pandas as pd
import asyncio
import aiohttp
from lxml import html as lxml_html
from tqdm import tqdm
import nest_asyncio
import numpy as np

# Apply nest_asyncio to enable nested running of asyncio
nest_asyncio.apply()

base_url = "https://www.law.go.kr/LSW/precInfoP.do?precSeq="

# Initialize empty lists for each column
case_info = []
case_ids = []
case_details = []
judgment_summaries = []
referenced_statutes = []
referenced_precedents = []
attorneys = []
orders = []
reasons = []
judges = []

async def fetch(ID, session):
    url = base_url + str(ID)
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        for i in tqdm(range(0, len(data), 80)):
            tasks = []
            for ID in data[i:i+80]:
                tasks.append(fetch(ID, session))
            htmls = await asyncio.gather(*tasks)
            for ID, html in zip(data[i:i+80], htmls):
                tree = lxml_html.fromstring(html)
                # Append the extracted data to the respective lists
                case_info.append(tree.xpath('//*[@id="contentBody"]/div[1]/text()'))
                case_ids.append(ID)

                # 판시사항
                case_details.append(tree.xpath('//*[@id="sa"]/following-sibling::p[@class="pty4"][1]//text()') if tree.xpath('//*[@id="sa"]') else np.nan)

                # 판결요지
                judgment_summaries.append(tree.xpath('//*[@id="yo"]/following-sibling::p[@class="pty4"][1]//text()') if tree.xpath('//*[@id="yo"]') else np.nan)

                # 참조조문
                referenced_statutes.append(tree.xpath('//*[@id="conLsJo"]/following-sibling::p[@class="pty4"][1]//text()') if tree.xpath('//*[@id="conLsJo"]') else np.nan)

                # 참조판례
                referenced_precedents.append(tree.xpath('//*[@id="conPrec"]/following-sibling::p[@class="pty4"][1]//text()') if tree.xpath('//*[@id="conPrec"]') else np.nan)

                orders.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-1]/text()'))
                reasons.append(tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()]/text()'))
                judges.append(tree.xpath('//*[@id="conScroll"]/div/text()') if tree.xpath('//*[@id="conScroll"]/div/text()') else np.nan)
                attorney_text = tree.xpath('//*[@id="conScroll"]/p[@class="pty4_dep1"][last()-3]/text()')
                keywords = ['변호사', '법무', '법인', '외']
                if attorney_text and any(keyword in ''.join(attorney_text) for keyword in keywords):
                    attorneys.append(attorney_text)
                else:
                    attorneys.append(np.nan)
            # Wait for 5 seconds
            await asyncio.sleep(5)

# Run the main function
await main()

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Case Info' : case_info,
    'Case ID': case_ids,
    'Case Details': case_details,
    'Judgment Summary': judgment_summaries,
    'Referenced Statutes': referenced_statutes,
    'Referenced Precedents': referenced_precedents,
    'Attorney': attorneys,
    'Order': orders,
    'Reason': reasons,
    'Judge': judges
})

df


100%|██████████| 92/92 [14:51<00:00,  9.69s/it]


,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge
0,[\n [대법원 2024. 2. 15. 선고 2023도15164 판결]...,239459,"[ [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입죄...",[ [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의 ...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항, ...","[ [1] , 대법원 2009. 9. 10. 선고 2009도4335 판결, (공20...",NaN,"[ 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 피고인은, ① 2...",[\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ]
1,[\n [대법원 2024. 2. 8. 선고 2023도16595 판결]\...,239467,[ [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판단...,[ [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성요...,"[ [1] , 형법 제319조 제1항, , [2] , 형법 제319조 제1항]","[ [1] , 대법원 2022. 3. 24. 선고 2017도18272 전원합의체 판...",[ 변호사 장원철],"[ 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을 ...","[ 상고이유를 판단한다. , 1. 2021. 9. 7. 건조물침입 부분 공소사실...",[\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...
2,[\n [대법원 2024. 1. 25. 선고 2020도10330 판결]...,238463,[ 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당사...,[ 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 재...,[형법 제347조 제1항],"[대법원 2004. 6. 25. 선고 2003도7124 판결, (공2004하, 12...",[ 법무법인 청률 담당변호사 김태창],"[ 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 이 사건 공소사실의 요지, 공소외인은 피고인 ...",[\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...
3,[\n [대법원 2024. 1. 4. 선고 2022도699 판결]\n ...,239243,"[ [1] , 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항, ...","[ [1] , 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항, 은...","[ [1] , 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항, ,...","[ [1][2] , 대법원 2020. 12. 10. 선고 2020도11471 판결,...",[ 법무법인 별 외 5인],[ 상고를 모두 기각한다.],"[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. ‘Bad Fath...",[\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ]
4,[\n [대법원 2024. 1. 4. 선고 2023도11313 판결]\...,239199,"[ [1] 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 , 형법 제238조, ...",[ [1] 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용하...,"[ [1] , 형법 제238조, , [2] , 형법 제238조]","[ [1] , 대법원 1995. 9. 5. 선고 95도1269 판결, (공1995하...",[ 법무법인 더킴로펌 담당변호사 김형석 외 1인],"[ 원심판결을 파기하고, 사건을 창원지방법원에 환송한다.]","[ 상고이유를 판단한다. , 1. 공소사실의 요지 , 가. 피고인은 2020...",[\n 대법관 이동원(재판장) 천대엽 권영준(주심) \n ]
...,...,...,...,...,...,...,...,...,...,...
7341,[\n [대법원 1953. 11. 23. 선고 4286형상162 판결]...,215365,[ 군정법령 제5호의 내용이 신형법과 저촉되는 경우],"[ 폭발물 소지에 관하여는 , 신형법 제121조, 와 군정법령 제5호 제2조와의 내...","[ 군정법령제5호 제2조, , 형법 제121조, , , 국가보안법 제3조]",NaN,NaN,[ 원판결을 파훼하고 본건을 서울고등법원에 환송한다.],"[ 피고인의 변호인 정근영 상고취의는 원심판결은 , 소정의 이유로서 파훼를 불면일...",[\n 대법관 김병로(재판장) 김세완 김갑수 허진 배정현 \n ...
7342,[\n [대법원 1953. 11. 9. 선고 4286형상181 판결]\...,215291,"[가. 간첩예비와 의율, 나. 간첩행위의 해석, 다. 군법회의 확정판결의 해석]","[가. 간첩예비죄에 , 구 형법 제88조, 만을 인용하여도 , 동법 제85조, 는 ...","[국가보안법 제3조, , , 구 형법 제85조 제1항, , , 제87조, , , 제...",NaN,NaN,[ 본건 상고를 기각한다.],"[피고인 1, 피고인 2, 피고인 3에 대한 검사 김홍수의 상고취의는 원판결은 피고...",[\n 대법관 김병로(재판장) 김세완 김갑수 허진 고재호 \n ...
7343,[\n [대법원 1952. 5. 27. 선고 4284형상76 판결]\n...,188581,[ 행위시와 재판시간에 수차 법령의 개폐에 의한 형의 변경과 법령 적용],[ 범죄행위 당시와 재판시의 중간에 수차 법령의 개폐로 인하여 형의 변경이 있는 때...,"[형법 제6조, , , , , 제10조, , 법령제127호 제2조 처벌, 법률 ...",NaN,NaN,[ 원판결을 파훼하고 본건을 서울고등법원에 차려한다.],[ 검사 박팔천의 상고이유는 원판결은 「범죄사실 급 증거설시는 원판결 적시사실과 동...,NaN
7344,[\n [대법원 1951. 5. 1. 선고 4283형상73 판결]\n ...,85840,[ 방화살인과 견련범],"[형법 제54조 후단, 에 소위 범죄의 수단이라 함은 어느 범죄의 성질상 그 수단으...","[형법 제54조, , , , 제108조, , , , 제198조]",NaN,NaN,[ 원판결은 파훼하고 본건을 서울고등법원에 차려하다.],[ 피고인 1의 변호인 양병오의 상고이유는 원판결은 의율착오의 위법을 면치 못할 것...,NaN


# 다시시작

In [40]:
df = pd.read_csv('/home/kannakawai9/NLP_single_project/law_data_dl/data_crawling/law_case.csv')
df.head()

,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge
0,['\n [대법원 2024. 2. 15. 선고 2023도15164 판결...,239459,"[' [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입...",[' [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의...,"[' [1] ', '형법 제319조 제1항', ' ', '[2] ', '형법 제31...","[' [1] ', '대법원 2009. 9. 10. 선고 2009도4335 판결', ...",NaN,"[' 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 이 사건 공소사실의 요지', ...",['\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ...
1,['\n [대법원 2024. 2. 8. 선고 2023도16595 판결]...,239467,[' [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판...,[' [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성...,"[' [1] ', '형법 제319조 제1항', ' ', '[2] ', '형법 제31...","[' [1] ', '대법원 2022. 3. 24. 선고 2017도18272 전원합의...",[' 변호사 장원철'],"[' 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을...","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 2021. 9. 7. 건조물침...",['\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...
2,['\n [대법원 2024. 1. 25. 선고 2020도10330 판결...,238463,[' 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당...,[' 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 ...,['형법 제347조 제1항'],"['대법원 2004. 6. 25. 선고 2003도7124 판결', '(공2004하,...",[' 법무법인 청률 담당변호사 김태창'],"[' 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 이 사건 공소사실의 요지', ...",['\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...
3,['\n [대법원 2024. 1. 4. 선고 2022도699 판결]\n...,239243,"[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1][2] ', '대법원 2020. 12. 10. 선고 2020도11471 ...",[' 법무법인 별 외 5인'],[' 상고를 모두 기각한다.'],"[' 상고이유를 판단한다.\xa0', ' 1.\xa0 공소사실의 요지\xa0', '...",['\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ...
4,['\n [대법원 2024. 1. 4. 선고 2023도11313 판결]...,239199,"[' [1] 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 ', '형법 제238...",[' [1] 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용...,"[' [1] ', '형법 제238조', ' ', '[2] ', '형법 제238조']","[' [1] ', '대법원 1995. 9. 5. 선고 95도1269 판결', '(공...",[' 법무법인 더킴로펌 담당변호사 김형석 외 1인'],"[' 원심판결을 파기하고, 사건을 창원지방법원에 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 공소사실의 요지\xa0', '...",['\n 대법관 이동원(재판장) 천대엽 권영준(주심) \n ...


In [41]:
len(df)

7346

In [43]:
df.columns

Index(['Case Info', 'Case ID', 'Case Details', 'Judgment Summary',
       'Referenced Statutes', 'Referenced Precedents', 'Attorney', 'Order',
       'Reason', 'Judge'],
      dtype='object')

In [45]:
df['Case Info'][0]

"['\\n        [대법원 2024. 2. 15. 선고 2023도15164 판결]\\n       ']"

In [49]:
from copy import deepcopy

In [86]:
temp = deepcopy(df)

In [87]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# 정규표현식 패턴 정의
date_pattern = r'\d{4}\.\s*\d{1,2}\.\s*\d{1,2}'
case_number_pattern = r'\d{4}도\d+'

# 'Case Info' 열에서 날짜와 사건번호 추출
temp['선고일자'] = temp['Case Info'].apply(lambda x: re.search(date_pattern, x).group(0) if re.search(date_pattern, x) else np.nan)
temp['사건번호'] = temp['Case Info'].apply(lambda x: re.search(case_number_pattern, x).group(0) if re.search(case_number_pattern, x) else np.nan)

# '선고일자' 열을 datetime 형식으로 변환
temp['선고일자'] = pd.to_datetime(temp['선고일자'], format='%Y. %m. %d')

In [88]:
temp.head()

,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge,선고일자,사건번호
0,['\n [대법원 2024. 2. 15. 선고 2023도15164 판결...,239459,"[' [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입...",[' [1] 다가구용 단독주택이나 다세대주택·연립주택·아파트와 같은 공동주택 내부의...,"[' [1] ', '형법 제319조 제1항', ' ', '[2] ', '형법 제31...","[' [1] ', '대법원 2009. 9. 10. 선고 2009도4335 판결', ...",NaN,"[' 원심판결을 파기하고, 사건을 서울북부지방법원으로 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 이 사건 공소사실의 요지', ...",['\n 대법관 이동원(재판장) 김상환 권영준(주심) \n ...,2024-02-15,2023도15164
1,['\n [대법원 2024. 2. 8. 선고 2023도16595 판결]...,239467,[' [1] 주거침입죄의 구성요건적 행위인 ‘침입’의 의미 및 침입에 해당하는지 판...,[' [1] 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다. 주거침입죄의 구성...,"[' [1] ', '형법 제319조 제1항', ' ', '[2] ', '형법 제31...","[' [1] ', '대법원 2022. 3. 24. 선고 2017도18272 전원합의...",[' 변호사 장원철'],"[' 원심판결 중 2021. 9. 7. 건조물침입 부분을 파기하고, 이 부분 사건을...","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 2021. 9. 7. 건조물침...",['\n 대법관 노태악(재판장) 김선수 오경미 서경환(주심) \n ...,2024-02-08,2023도16595
2,['\n [대법원 2024. 1. 25. 선고 2020도10330 판결...,238463,[' 소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 / 소송당...,[' 소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 ...,['형법 제347조 제1항'],"['대법원 2004. 6. 25. 선고 2003도7124 판결', '(공2004하,...",[' 법무법인 청률 담당변호사 김태창'],"[' 원심판결을 파기하고, 사건을 서울동부지방법원에 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 이 사건 공소사실의 요지', ...",['\n 대법관 서경환(재판장) 김선수 노태악(주심) 오경미 \n ...,2024-01-25,2020도10330
3,['\n [대법원 2024. 1. 4. 선고 2022도699 판결]\n...,239243,"[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1] ', '정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항...","[' [1][2] ', '대법원 2020. 12. 10. 선고 2020도11471 ...",[' 법무법인 별 외 5인'],[' 상고를 모두 기각한다.'],"[' 상고이유를 판단한다.\xa0', ' 1.\xa0 공소사실의 요지\xa0', '...",['\n 대법관 권영준(재판장) 이동원 천대엽(주심) \n ...,2024-01-04,2022도699
4,['\n [대법원 2024. 1. 4. 선고 2023도11313 판결]...,239199,"[' [1] 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 ', '형법 제238...",[' [1] 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용...,"[' [1] ', '형법 제238조', ' ', '[2] ', '형법 제238조']","[' [1] ', '대법원 1995. 9. 5. 선고 95도1269 판결', '(공...",[' 법무법인 더킴로펌 담당변호사 김형석 외 1인'],"[' 원심판결을 파기하고, 사건을 창원지방법원에 환송한다.']","[' 상고이유를 판단한다.\xa0', ' 1.\xa0 공소사실의 요지\xa0', '...",['\n 대법관 이동원(재판장) 천대엽 권영준(주심) \n ...,2024-01-04,2023도11313


In [81]:
df['Case Details'][0]

"[' [1] 공동주택 내부의 엘리베이터, 공용 계단, 복도 등 공용 부분이 주거침입죄의 객체인 ‘사람의 주거’에 해당하는지 여부', '(적극)', ' / 거주자가 아닌 외부인이 공동주택의 공용 부분에 출입한 것이 공동주택 거주자들에 대한 주거침입에 해당하는지 판단하는 기준', ' [2] 주거에 들어가는 행위가 거주자의 의사에 반한다는 주관적 사정이 주거침입죄에서의 침입행위에 해당하는지를 판단할 때 고려할 하나의 요소인지 여부', '(적극)', ' 및 이때 그 고려의 정도는 주거 등의 형태와 용도·성질, 외부인에 대한 출입의 통제·관리 방식과 상태 등 출입 당시 상황에 따라 달리 평가되는지 여부', '(적극)', ' [3] 피고인이 예전 여자친구인 甲의 사적 대화 등을 몰래 녹음하거나 현관문에 甲에게 불안감을 불러일으킬 수 있는 문구가 기재된 마스크를 걸어놓거나 甲이 다른 남자와 찍은 사진을 올려놓으려는 의도로 3차례에 걸쳐 야간에 甲이 거주하는 빌라 건물의 공동현관, 계단을 통해 甲의 2층 주거 현관문 앞까지 들어간 사안에서, 빌라 건물 공용 부분의 성격, 외부인의 무단출입에 대한 통제·관리 방식과 상태, 피고인과 甲의 관계, 피고인의 출입 목적 및 경위와 출입 시간, 출입행위를 전후한 피고인의 행동, 甲의 의사와 행동, 주거공간 무단출입에 관한 사회 통념 등 제반 사정을 종합하면, 피고인은 甲 주거의 사실상 평온상태를 해치는 행위태양으로 빌라 건물에 출입하였다고 볼 여지가 충분하다는 이유로, 이와 달리 본 원심판결에 주거침입죄의 ‘침입’에 관한 법리오해 등의 잘못이 있다고 한 사례']"

In [82]:
import pandas as pd
import numpy as np
import re

# 쉬운 한글로 변환하는 함수
def hanja_to_hangul(text):
    # 그에 해당하는 한글을 딕셔너리 형태로 저장
    hanja_to_hangul_dict = {
        '적극': '적극적으로 인정한다',
        '소극': '그렇다고 보기는 힘든 면이 있다'
    }
    # 딕셔너리를 순회하며 해당하는 한자를 한글로 변환
    for hanja, hangul in hanja_to_hangul_dict.items():
        text = text.replace(hanja, hangul)
    return text

# 특수문자를 제거하는 함수
def remove_special_characters(text):
    # 특수문자를 제거하는 정규표현식 패턴
    pattern = r'[^\w\s]'
    text = re.sub(pattern, '', text)
    return text

# 'Case Details' 열에서 텍스트를 처리
temp['Case Details'] = temp['Case Details'].apply(lambda x: hanja_to_hangul(remove_special_characters(' '.join(str(i) for i in str(x).split()))))


In [83]:
temp['Case Details'][0]

' 1 공동주택 내부의 엘리베이터 공용 계단 복도 등 공용 부분이 주거침입죄의 객체인 사람의 주거에 해당하는지 여부 적극적으로 인정한다   거주자가 아닌 외부인이 공동주택의 공용 부분에 출입한 것이 공동주택 거주자들에 대한 주거침입에 해당하는지 판단하는 기준  2 주거에 들어가는 행위가 거주자의 의사에 반한다는 주관적 사정이 주거침입죄에서의 침입행위에 해당하는지를 판단할 때 고려할 하나의 요소인지 여부 적극적으로 인정한다  및 이때 그 고려의 정도는 주거 등의 형태와 용도성질 외부인에 대한 출입의 통제관리 방식과 상태 등 출입 당시 상황에 따라 달리 평가되는지 여부 적극적으로 인정한다  3 피고인이 예전 여자친구인 甲의 사적 대화 등을 몰래 녹음하거나 현관문에 甲에게 불안감을 불러일으킬 수 있는 문구가 기재된 마스크를 걸어놓거나 甲이 다른 남자와 찍은 사진을 올려놓으려는 의도로 3차례에 걸쳐 야간에 甲이 거주하는 빌라 건물의 공동현관 계단을 통해 甲의 2층 주거 현관문 앞까지 들어간 사안에서 빌라 건물 공용 부분의 성격 외부인의 무단출입에 대한 통제관리 방식과 상태 피고인과 甲의 관계 피고인의 출입 목적 및 경위와 출입 시간 출입행위를 전후한 피고인의 행동 甲의 의사와 행동 주거공간 무단출입에 관한 사회 통념 등 제반 사정을 종합하면 피고인은 甲 주거의 사실상 평온상태를 해치는 행위태양으로 빌라 건물에 출입하였다고 볼 여지가 충분하다는 이유로 이와 달리 본 원심판결에 주거침입죄의 침입에 관한 법리오해 등의 잘못이 있다고 한 사례'

In [84]:
import pandas as pd
import numpy as np
import re

# 데이터프레임의 모든 열을 순회
for col in temp.columns:
    if col != '선고일자' and col != '사건번호':

        # 각 열의 값에 대해 문자열로 변환
        temp[col] = temp[col].astype(str)
    
        # 특수문자를 제거하고 연속된 공백을 하나의 공백으로 바꾸는 함수
        def clean_text(text):
            # 특수문자를 제거하는 정규표현식 패턴
            pattern = r'[^\w\s]'
            text = re.sub(pattern, '', text)
        
            # 연속된 공백을 하나의 공백으로 바꾸는 정규표현식 패턴
            pattern = r'\s{2,}'
            text = re.sub(pattern, ' ', text)
            return text

    # 각 열의 값에 대해 텍스트를 처리
        temp[col] = temp[col].apply(clean_text)


In [85]:
temp.head()

,Case Info,Case ID,Case Details,Judgment Summary,Referenced Statutes,Referenced Precedents,Attorney,Order,Reason,Judge,선고일자,사건번호
0,n 대법원 2024 2 15 선고 2023도15164 판결n,239459,1 공동주택 내부의 엘리베이터 공용 계단 복도 등 공용 부분이 주거침입죄의 객체인...,1 다가구용 단독주택이나 다세대주택연립주택아파트와 같은 공동주택 내부의 엘리베이터...,1 형법 제319조 제1항 2 형법 제319조 제1항 3 형법 제319조 제1항,1 대법원 2009 9 10 선고 2009도4335 판결 공2009하 1705 대...,nan,원심판결을 파기하고 사건을 서울북부지방법원으로 환송한다,상고이유를 판단한다xa0 1xa0 이 사건 공소사실의 요지 피고인은 ① 2021 ...,n 대법관 이동원재판장 김상환 권영준주심 n,2024-02-15,2023도15164
1,n 대법원 2024 2 8 선고 2023도16595 판결n,239467,1 주거침입죄의 구성요건적 행위인 침입의 의미 및 침입에 해당하는지 판단하는 기준...,1 주거침입죄는 사실상 주거의 평온을 보호법익으로 한다 주거침입죄의 구성요건적 행...,1 형법 제319조 제1항 2 형법 제319조 제1항,1 대법원 2022 3 24 선고 2017도18272 전원합의체 판결 공2022상...,변호사 장원철,원심판결 중 2021 9 7 건조물침입 부분을 파기하고 이 부분 사건을 서울중앙지...,상고이유를 판단한다xa0 1xa0 2021 9 7 건조물침입 부분 공소사실의 요지...,n 대법관 노태악재판장 김선수 오경미 서경환주심 n,2024-02-08,2023도16595
2,n 대법원 2024 1 25 선고 2020도10330 판결n,238463,소송사기죄 적용의 엄격성 및 소송사기를 유죄로 인정할 수 있는 경우 소송당사자들이...,소송사기는 법원을 속여 자기에게 유리한 판결을 얻음으로써 상대방의 재물 또는 재산...,형법 제347조 제1항,대법원 2004 6 25 선고 2003도7124 판결 공2004하 1277,법무법인 청률 담당변호사 김태창,원심판결을 파기하고 사건을 서울동부지방법원에 환송한다,상고이유를 판단한다xa0 1xa0 이 사건 공소사실의 요지 공소외인은 피고인 1이...,n 대법관 서경환재판장 김선수 노태악주심 오경미 n,2024-01-25,2020도10330
3,n 대법원 2024 1 4 선고 2022도699 판결n,239243,1 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항 명예훼손죄의 ...,1 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항 은 사람을 비...,1 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제70조 제1항 형사소송법 제...,12 대법원 2020 12 10 선고 2020도11471 판결 공2021상 253...,법무법인 별 외 5인,상고를 모두 기각한다,상고이유를 판단한다xa0 1xa0 공소사실의 요지xa0 가xa0 Bad Fathe...,n 대법관 권영준재판장 이동원 천대엽주심 n,2024-01-04,2022도699
4,n 대법원 2024 1 4 선고 2023도11313 판결n,239199,1 형법상 인장에 관한 죄에서 인장과 기호의 의미 및 형법 제238조 의 공기호에...,1 형법상 인장에 관한 죄에서 인장은 사람의 동일성을 표시하기 위하여 사용하는 일...,1 형법 제238조 2 형법 제238조,1 대법원 1995 9 5 선고 95도1269 판결 공1995하 3458,법무법인 더킴로펌 담당변호사 김형석 외 1인,원심판결을 파기하고 사건을 창원지방법원에 환송한다,상고이유를 판단한다xa0 1xa0 공소사실의 요지xa0 가xa0 피고인은 2020...,n 대법관 이동원재판장 천대엽 권영준주심 n,2024-01-04,2023도11313
